## LQTMOMENT Tutorial 3: Calculating Moment Magnitudes from a Batch of Earthquakes

There is no limit on how many earthquakes `lqtmoment` can process in a single batch to calculate the moment magnitude automatically in one run, as long as provided catalog input, waveform directory structure, calibration file are well-defined and follows the `lqtmoment` format.

> **⚠️ CAUTION ⚠️**
> 
> Before continuing this tutorial, make sure you have already seen the 1st and 2nd LQT tutorials.

### 1. Programmatic Approach

#### A. Import Moment Magnitude Estimator Module

For calculating a batch of earthquakes data we need to import `magnitude_estimator` function from `lqtmoment` package.

In [11]:
from lqtmoment import magnitude_estimator
from pathlib import Path

#### B. Initialize Input/Output File/Dir

In [ ]:
# Initialize directories object
dirs = {
    "wave_dir": r"..\tests\sample_tests_data\data\wave",
    "calib_dir": r"..\tests\sample_tests_data\data\calibration",
    "catalog_file": r"..\tests\sample_tests_data\results\lqt_catalog\lqt_catalog.xlsx",
    "config_file": r"..\tests\sample_tests_data\calculation configuration\config_test.ini",
    "figures_dir": r"..\tests\sample_tests_data\figures",
    "output_dir": r"..\tests\sample_tests_data\results\calculation"
}

In the `dirs` dictionary object, we specify the absolute paths for directories and input files including (`wave directory`, `calibration/response file directory`, `lqtmoment format catalog`, `configuration file`) and also the outputs (`output figure directory`, `calculation output directory`).

#### C. Run The Magnitude Estimator

In this programmatic approach, the magnitude estimator returns three `Pandas DataFrame` objects, the complete lqt catalog merged with magnitude results (there will be new column called 'magnitude' in lqt catalog), detail calculation result and the detailed fitting result for all earthquakes in the batch.

In [ ]:
merged_lqt_catalog, lqt_moment_result, lqt_fitting_result = magnitude_estimator(    
                                                            wave_dir= dirs['wave_dir'],
                                                            cal_dir= dirs['calib_dir'],
                                                            catalog_file= dirs['catalog_file'],
                                                            config_file= dirs['config_file'],
                                                            id_start=1001,
                                                            id_end=1005,
                                                            lqt_mode=True,
                                                            generate_figure=True,
                                                            fig_dir= dirs['figures_dir'],
                                                            save_output_file= True,
                                                            output_dir= dirs['output_dir']
                                                            )

> **ℹ️ INFO ℹ️**
> 
> If `id_start` and `id_end` are not specified by the user, the `lqtmoment` will automatically set the `id_start` to the minimum earthquake ID  and `id_end` to the maximum earthquake ID in the catalog, respectively.
> 
> If `lqt_mode` set to `False`, for the `very_local_earthquake` category, `lqtmoment` will perform calculations using the ZRT component systems, which is simpler and reduces runtime. 
>
> The `generate_figure` parameters defaults to `False` if not specified by the user.
>
> If `save_output_file` is set to `True`, `lqtmoment` will automatically save the result to the `output_dir` in the `.xlsx` format unless the user specifies otherwise (`.csv`). If not set to `True`, as the result, the function will only returns three `Pandas` DataFrame that can be used later by the User for analysis and assessment.


### 2. Command-Line Interface Approach

When calculating moment magnitude, `lqtmoment` also offers **Command-Line Interface (CLI)** functionality. If the input and output directories follow `lqtmoment` formats correctly,  you can easily perform the moment magnitude calculation by entering command line in your terminal, as shown bellow (ensure that the `lqtmoment` package is correctly installed in your working environment beforehand):

In [ ]:
$ lqtmoment --wave-dir ..\tests\sample_tests_data\data\wave --cal-dir ..\tests\sample_tests_data\data\calibration --catalog-file ..\tests\sample_tests_data\results\lqt_catalog\lqt_catalog.xlsx --config-file ..\tests\sample_tests_data\calculation configuration\config_test.ini --id-start 1001 --id-end 1005  --create-figure --fig-dir ..\tests\sample_tests_data\figures --output-dir ..\tests\sample_tests_data\results\calculation

> **ℹ️ INFO ℹ️**
> 
> In this **CLI** approach, if you don’t specify the ID range, `lqtmoment` will handle it automatically. To disable `lqt_mode` for `very_local_earthquakes` category, you can use the `--non-lqt` argument.




### 3. Understanding the configuration (.INI) and velocity_model (.JSON) file.

#### A. The Configuration File

As shown in the tutorial above, you also need to prepare the configuration file `config.ini` and the velocity model file `velocity_model.json` to run the program properly for your specific case. The `config.ini` file contains important parameters required to run `lqtmoment`, so it's essential to configure it correctly. The details of the `config.ini` file are as follows (note that the parameters shown are just examples):
```ini
    [Wave]
    snr_threshold = 1.25
    water_level = 20.0
    pre_filter = 0.1,0.15,100,125
    apply_post_instrument_removal_filter = True
    post_filter_f_min = 3
    post_filter_f_max = 60
    trim_mode = dynamic
    sec_bf_p_arr_trim = 5
    sec_af_p_arr_trim = 25
    padding_before_arrival = 0.1
    min_p_window = 1.0
    max_p_window = 10.0
    min_s_window = 2.0
    max_s_window = 20.0
    noise_duration = 0.5
    noise_padding = 0.2

    [Magnitude]
    r_pattern_p = 0.52
    r_pattern_s = 0.63
    free_surface_factor = 2.0
    k_p = 0.32
    k_s = 0.21
    mw_constant = 6.07
    taup_model = iasp91
    velocity_model_file = velocity_model.json

    [Spectral]
    f_min = 1
    f_max = 50
    omega_0_range_min = 0.001
    omega_0_range_max = 1000
    q_range_min = 50
    q_range_max = 250
    fc_range_buffer = 1
    default_n_samples = 3000
    n_factor = 2
    y_factor = 1

    [Performance]
    use_parallel = False
    logging_level = INFO
```

As you can see, there are four essential sections in the parameter settings. First, you need to set the parameters related to your waveform/seismogram data. Next are the parameters for moment magnitude calculation, followed by the spectral fitting parameters, and finally, the performance-related parameters.

> **⚠️ CAUTION ⚠️**
> 
> You need to follow and fill in all of these parameter fields; otherwise, the program will fall back to the default parameters in `lqtmoment`, which are likely not suitable for your specific case.
>

The detailed explanation for each of the parameters mentioned above is as follows:

#### A. Wave Part
This `Wave` section handles all the necessary steps to prepare your raw seismogram for simulating the source spectra, performing spectral fitting, and ultimately calculating the moment magnitude.

Detailed Parameters:

- **snr_threshold:** Minimum signal-to-noise ratio for trace acceptance (unitless, default: 1.5)
- **water_level:** Water level for deconvolution stabilization (unitless, default: 60)
- **pre_filter:** Pre-processing bandpass filter corners (f1,f2,f3,f4 in Hz, default: 0.1,0.2,55,60)
- **apply_post_instrument_removal_filter:** If True, post filter after instrument removal will be applied (default: True).
- **post_filter_f_min:** Post-processing filter minimum frequency (Hz, default: 3)
- **post_filter_f_max:** Post-processing filter maximum frequency (Hz, default: 50)
- **trim_mede:** Mode used to trim, 'dynamic' or 'static', if 'dynamic' consider the coda_time in catalog.
- **sec_bf_p_arr_trim:** seconds before P arrival trim to start the trimming.
- **sec_af_p_arr_trim:** seconds after P arrival trim to end the trimming.
- **padding_before_arrival:** Padding before P/S arrival for signal window (seconds, default: 0.1).
- **min_p_window:** Minimum P phase window in second for calculating source spectra (default: 1.0).
- **max_p_window:** Maximum P phase window in second for calculating source spectra (default: 10.0).
- **min_s_window:** Minimum S phase window in second for calculating source spectra (default: 2.0).
- **max_s_window:** Maximum S phase window in second for calculating source spectra (default: 20.0).
- **noise_duration:** Duration of noise window (seconds, default: 0.5)
- **noise_padding:** Padding around noise window (seconds, default: 0.2)

> **⚠️ CAUTION ⚠️**
> 
> If you rely more on `pre-filter` before performing instrument response removal, you can simply set the `water-level` to `None`. This means that no numerical stabilization mechanism will be applied when dividing the raw seismogram data by the instrument response in the frequency domain. If you'd like to learn more about what the `water-level` parameter actually does, you can visit this resource [water-level](http://eqseis.geosc.psu.edu/cammon/HTML/RftnDocs/seq01.html).
>
> The parameters `apply_post_instrument_removal_filter`, `post_filter_f_min`, and `post_filter_f_max` should be considered if you want to apply post-instrument removal filtering. This means you're applying a filter to seismogram data that has already been converted to true ground displacement. This approach is beneficial because applying a `pre-filter` requires extra caution to avoid introducing instability during the deconvolution process—so you typically don't want to set it too strictly. By applying a post-filter afterward, you can enhance the signal-to-noise ratio (SNR) while also gaining more control over the final frequency range that matters for your analysis. If you don't want to do this, just set the `apply_post_instrument_removal_filter` to `false` and the`post_filter_f_min`, and `post_filter_f_max` will be neglected. 
>
> In `lqtmoment` a trimming procedure is applied to shorten your raw seismogram data to improve computation time so that it only includes the portion necessary for calculating moment magnitude. If you set the trimming mode to `dynamic`, program will use the `coda_time` provided in your catalog. If no `coda_time` is available, it will automatically fall back to the static value defined in `sec_af_p_arr_trim`. On the other hand, if you set `trim_mode` to `static`, the `coda_time` will be ignored even if it exists in your lqt catalog.
>
> Phases windowing is internal dynamic process within `lqtmoment`. The program ensures there is no phase contamination and minimizes the inclusion of unwanted signals in the calculation of source spectra. Users generally set the parameters `min_p_window`, `max_p_window`, `min_s_window`, and `max_s_window` according to their specific cases. Internally, `lqtmoment` aggregates these parameters with the `s_p_lag_time` setting to enable more robust windowing.



#### B. Magnitude Part
This `Magnitude` section handles all parameters necessary to calculate the moment magnitude.

Detailed Parameters:

- **r_pattern_p:** Radiation pattern correction for P-waves (unitless, default: 0.52, Aki & Richards, 2002)
- **r_pattern_s:** Radiation pattern correction for S-waves (unitless, default: 0.63, Aki & Richards, 2002)
- **free_surface_factor:** Free surface amplification factor (unitless, default: 2.0)
- **k_p:** Geometric spreading factor for P-waves (unitless, default: 0.32)
- **k_s:** Geometric spreading factor for S-waves (unitless, default: 0.21)
- **mw_constant:** The empirical constant value for moment magnitude calculation
- **taup_model:** 1-D velocity model for estimating incident angle and source distance for regional to teleseismic earthquake (default: 'iasp91')
- **velocity_model_file:** Path to a JSON file defining the velocity model(default: None, uses built-in model)

> **⚠️ CAUTION ⚠️**
> 
> In the `velocity_model_file` parameter, you specify the path to the `.json` file containing your 1-D velocity model. You can use either an absolute or relative path. If this file is not specified, the velocity parameters will fall back to the default values used by the `lqtmoment` program, which may not be suitable for your specific case.
>

Here is an example of what the `.json` file should look like:

```json
    {
        "layer_boundaries": [ [-3.00,-1.90],[-1.90,-0.59],[-0.59, 0.22],[0.22, 2.50], [2.50, 7.00], [7.00,9.00] , [9.00,15.00], [15.00,33.00], [33.00,9999]],
        "velocity_vp": [2.68, 2.99, 3.95, 4.50, 4.99, 5.60, 5.80, 6.40, 8.00],
        "velocity_vs": [1.60, 1.79, 2.37, 2.69, 2.99, 3.35, 3.47, 3.83, 4.79],
        "density": [ 2375, 2375, 2375, 2465, 2529, 2750, 2750, 2931, 2931]
    }
```

#### C. Spectral Fitting Part
This `Spectral` section handles all parameters necessary to perform spectral fitting.

Detailed Parameters:

- **f_min:** Minimum frequency for spectral fitting (Hz, default: 1)
- **f_max:** Maximum frequency for spectral fitting (Hz, default: 40)
- **omega_0_range_min:** Minimum Omega_0 for fitting (nm/Hz, default: 0.01) [Confirm units]
- **omega_0_range_max:** Maximum Omega_0 for fitting (mm/Hz, default: 1000) [Confirm units]
- **q_range_min:** Minimum Q factor for attenuation (unitless, default: 50)
- **q_range_max:** Maximum Q factor for attenuation (unitless, default: 250)
- **fc_range_buffer:** Buffer factor for corner frequency range (unitless, default: 1)
- **default_n_samples:** Number of Monte Carlo samples for fitting (default: 3000)
- **n_factor:** Stress drop exponent in Brune model (default: 2, Brune, 1970)
- **y_factor:** High-frequency fall-off exponent (default: 1, Brune; 2 for Boatwright)

> **⚠️ CAUTION ⚠️**
> 
> A higher value for `default_n_samples` will result in increased computational time.
>

#### D. Computation Performance Part
And the last, `Performance` section handles `lqtmoment` computation performance .

Detailed Parameters:

- **use_parallel:** Enable parallel processing (default: false)
- **logging_level:** Logging verbosity (DEBUG, INFO, WARNING, ERROR, default: INFO)

> **⚠️ CAUTION ⚠️**
> 
> In this version, `lqtmoment` does not support parallel computing. This feature is planned to be implemented in a future release once the program reaches a stable stage.
>